## Классификация текста с использованием CNN
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1NttEyPGuhurOdJhOZu5fRC-NaoGqrx-Z)

Используя данные отзывов IMDB, построим CNN для классификации документов на позитивный и негативный классы.

Источник изложения: https://github.com/bentrevett/pytorch-sentiment-analysis

В предположении, что PyTorch уже установлен, поставим дополнительные модули и загрузим модель для токенизации:

In [1]:
# !pip3 install https://download.pytorch.org/whl/cpu/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl

In [0]:
!pip install torchvision

     |████████████████████████████████| 748.9MB 28kB/s 
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2


In [0]:
!pip install torchtext

In [0]:
!pip install spacy

In [2]:
#!python3.6 -m spacy download en
!python -m spacy download en_core_web_sm

/usr/bin/python: No module named spacy


In [3]:
import spacy
#import en
en_nlp = spacy.load('en_core_web_sm')

Загрузим датасет и получим из него выборку:

In [5]:
import torch
from torchtext import data
from torchtext import datasets

print(torch.__version__)

SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField()

train_src, test = datasets.IMDB.splits(TEXT, LABEL)

Попробуем обучить простую CNN на векторах слов. С учётом того, что в коллекции 100К уникальных слов, и векторы получатся достаточно громоздкие, урежем коллекцию до 25К слов, для всех прочих заведя токен unk (unknown). Кроме того, разобьём обучающий сет на обучение и валидацию для настройки параметров.

In [0]:
import random
train, valid = train_src.split(random_state=random.seed(SEED))

In [0]:
TEXT.build_vocab(train, max_size=25000)
LABEL.build_vocab(train)

Выше в словаре, помимо отсечения 25К наиболее частых слов, также появятся два специальных токена - unk и pad (padding).

Теперь создадим батчи, предварительно отсортировав тексты по длине, что минимизировать вставки pad-токенов:

In [0]:
BATCH_SIZE = 16

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.text), 
    repeat=False)

Опишем функцию подсчёта accuracy, а также функции обучения и применения сети:

In [0]:
import torch.nn.functional as F

def binary_accuracy(preds, y):
    rounded_preds = torch.round(F.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train_func(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        
        predictions = model(batch.text.cuda()).squeeze(1)

        loss = criterion(predictions.float(), batch.label.float().cuda())
        acc = binary_accuracy(predictions.float(), batch.label.float().cuda())
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss
        epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate_func(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text.cuda()).squeeze(1)

            loss = criterion(predictions.float(), batch.label.float().cuda())
            acc = binary_accuracy(predictions.float(), batch.label.float().cuda())

            epoch_loss += loss
            epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Начнём с подготовки данных:

In [0]:
TEXT.build_vocab(train, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(train)


BATCH_SIZEBATCH_S  = 8

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.text), 
    repeat=False)

.vector_cache/glove.6B.zip: 862MB [00:44, 19.2MB/s]                           
100%|█████████▉| 399103/400000 [00:12<00:00, 31526.93it/s]

Для создания свёрточного слоя воспользуемся nn.Conv2d, in_channels в нашем случае один (текст), out_channels -- это число число фильтров и размер ядер всех фильтров. Каждый фильтр будет иметь размерность [n x размерность эмбеддинга], где n - размер обрабатываемой n-граммы.

Важно, что предложения имели длину не меньше размера самого большого из используемых фильтров (здесь это не страшно, поскольку в используемых данных нет текстов, состоящих из пяти и менее слов).

In [0]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[0], embedding_dim))
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[1], embedding_dim))
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[2], embedding_dim))
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        #x = [sent len, batch size]
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

Сейчас мы можем использовать только три различных фильтра, хотелось бы больше. Вообще, можно воспользоваться `nn.ModuleList`, чтобы создать слои списком и сделать так, чтобы фильтров создавалось по количеству элементов в filter_sizes. [(Как здесь).](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb)

Cоздадим модель и загрузим предобученные эмбеддинги:


In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

In [0]:
TEXT.build_vocab(train, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(train)

pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [0]:
BATCH_SIZE  = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.text), 
    repeat=False)

Используя определённые ранее функции, запустим обучение с оптимизатором Adam и оценим качество на валидации и тесте:

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.cuda()

In [0]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_func(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_func(model, valid_iterator, criterion)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 01, Train Loss: 0.503, Train Acc: 74.45%, Val. Loss: 0.343, Val. Acc: 85.63%
Epoch: 02, Train Loss: 0.309, Train Acc: 87.06%, Val. Loss: 0.307, Val. Acc: 87.02%
Epoch: 03, Train Loss: 0.219, Train Acc: 91.36%, Val. Loss: 0.312, Val. Acc: 87.30%
Epoch: 04, Train Loss: 0.148, Train Acc: 94.58%, Val. Loss: 0.290, Val. Acc: 88.78%
Epoch: 05, Train Loss: 0.088, Train Acc: 97.16%, Val. Loss: 0.304, Val. Acc: 89.29%


In [0]:
test_loss , test_acc = evaluate_func(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Test Loss: 0.322, Test Acc: 87.86%
